# **Sesión 7 - Random Forest**

In [3]:
#0. Importar las librerias
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split

#**************Librerias nuevas**************
from sklearn.preprocessing import MinMaxScaler # Escalar los datos -->  Valores entre 0 y 1
from sklearn.ensemble  import RandomForestClassifier # Permite utilizar algoritmo  "Bosque aleatorio"

In [ ]:
#1. Leer y Explorar los datos
# Descomprimir el archivo ZIP y cargar el CSV en un DataFrame
with zipfile.ZipFile('./Data/S7_Tarjetas_credito.zip') as z:
    with z.open('tarjetas_credito.csv') as f:
        df = pd.read_csv(f)

df.head(10)
#Transacciones con tarjeta de credito
#id, ubicación, fecha, hora, categoria(restaurant, farmacia, supermercado)
#V1,v2 ---> Forma anomima   PCA 
# Clase  0(T. legitima)  1(T. Fraudulenta)

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Monto,Clase
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [5]:
#2. Preprar datos
#Escalar datos  ?
#Crear el objeto escalador
escala = MinMaxScaler(feature_range=(0,1)) #Señor escalador
#Data Normada o escalada
normado = escala.fit_transform(df)
normado #esta algo feo  --> El resultado es un array

#Crear un DataFrame a partir del array
df_normado = pd.DataFrame(data=normado, columns=df.columns)
df_normado #Vamos a trabajar con  df_normado   ?   Buen rendimiento,  aprenda de la mejor forma  :)

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Monto,Clase
0,0.000000,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.000000,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.000006,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.000006,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.000012,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,0.999965,0.756448,0.873531,0.666991,0.160317,0.729603,0.236810,0.235393,0.863749,0.528729,...,0.564920,0.515249,0.680500,0.313600,0.658558,0.466291,0.433929,0.329840,0.000030,0.0
284803,0.999971,0.945845,0.766677,0.872678,0.219189,0.771561,0.273661,0.265504,0.788548,0.482925,...,0.564933,0.553153,0.665619,0.245298,0.543855,0.360884,0.417775,0.312038,0.000965,0.0
284804,0.999977,0.990905,0.764080,0.781102,0.227202,0.783425,0.293496,0.263547,0.792985,0.477677,...,0.565220,0.537005,0.664877,0.468492,0.592824,0.411177,0.416593,0.312585,0.002642,0.0
284805,0.999977,0.954209,0.772856,0.849587,0.282508,0.763172,0.269291,0.261175,0.792671,0.476287,...,0.565755,0.547353,0.663008,0.398836,0.545958,0.514746,0.418520,0.315245,0.000389,0.0


In [6]:
#3. Separar los datos   "Entrenamiento"    "Prueba"
#Variable con los datos descriptivos(variable independiente)
X = df_normado.drop("Clase", axis=1)
X
#Variable Predictiva(Dependiente)
y = df_normado["Clase"]
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
284802    0.0
284803    0.0
284804    0.0
284805    0.0
284806    0.0
Name: Clase, Length: 284807, dtype: float64

In [7]:
#ahora si separar los datos
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, train_size=0.7, random_state=42)
X_entrena

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Monto
2557,0.012182,0.919397,0.762191,0.851600,0.176286,0.771251,0.246327,0.271107,0.779574,0.508360,...,0.571407,0.558081,0.552703,0.678322,0.439229,0.595340,0.297794,0.423751,0.310125,0.000039
247823,0.889520,0.952964,0.724374,0.778523,0.283138,0.766401,0.278401,0.274808,0.784991,0.463325,...,0.603079,0.575233,0.505742,0.651530,0.269977,0.549912,0.531818,0.411152,0.315363,0.046486
152342,0.563006,0.927548,0.761271,0.876666,0.209360,0.757554,0.278267,0.256722,0.795076,0.530334,...,0.574367,0.553259,0.490206,0.661444,0.383598,0.604113,0.561932,0.415359,0.314911,0.002919
103385,0.397171,0.978550,0.769143,0.839865,0.272305,0.765380,0.261892,0.264357,0.787056,0.460452,...,0.578668,0.557603,0.475516,0.667168,0.331860,0.584950,0.446704,0.416198,0.313366,0.000077
8771,0.069164,0.941926,0.774648,0.856429,0.183325,0.770226,0.268183,0.267388,0.788291,0.510383,...,0.581569,0.556974,0.488097,0.663592,0.195089,0.564578,0.553733,0.422966,0.317511,0.001452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.437624,0.978230,0.768322,0.845914,0.272392,0.763721,0.260877,0.264349,0.786664,0.458846,...,0.578588,0.558481,0.483241,0.667831,0.381407,0.583040,0.443292,0.416125,0.313204,0.000077
259178,0.920181,0.945111,0.768778,0.817617,0.196913,0.778779,0.302352,0.262516,0.799981,0.476583,...,0.577117,0.562077,0.509118,0.669025,0.474244,0.486124,0.299517,0.417729,0.315905,0.003473
131932,0.461798,0.955803,0.777735,0.863833,0.273464,0.768068,0.254824,0.271915,0.781145,0.445855,...,0.580342,0.562328,0.526549,0.661109,0.436518,0.591231,0.360359,0.414275,0.309142,0.000153
146867,0.508884,0.908200,0.792106,0.793725,0.083528,0.778280,0.290444,0.262481,0.809173,0.482767,...,0.584710,0.556115,0.461164,0.667562,0.473711,0.595853,0.495085,0.425683,0.319939,0.000039


In [8]:
#4.  Crear al modelo  "Bosque aleatorio"  -- Muchos Árboles
bosque = RandomForestClassifier() #Crea 100 Árboles
#bosque = RandomForestClassifier(n_estimator=200)

#Modelo  recien nacio --- No conoce nada ...

In [9]:
#4.1.  entrenar al modelo
bosque.fit(X_entrena,y_entrena)

RandomForestClassifier()

In [10]:
#5. Si el modelo es confiable para realizar predicciones
bosque.score(X_prueba, y_prueba)
#Es apropiado para identificar si la transacción es o no fraudulento

0.9996137776061234

In [11]:
#6. Predecir si la transacción es Fraudulenta
nuevo_registro = pd.DataFrame({
    'Duracion': [0.000006], 'V1': [0.452345], 'V2': [0.564789], 'V3': [0.123456], 'V4': [0.654321],
    'V5': [0.987654], 'V6': [0.345678], 'V7': [0.234567], 'V8': [0.876543], 'V9': [0.456789],
    'V10': [0.567890], 'V11': [0.678901], 'V12': [0.789012], 'V13': [0.890123], 'V14': [0.901234],
    'V15': [0.012345], 'V16': [0.543210], 'V17': [0.432109], 'V18': [0.321098], 'V19': [0.210987],
    'V20': [0.109876], 'V21': [0.098765], 'V22': [0.887654], 'V23': [0.776543], 'V24': [0.665432],
    'V25': [0.554321], 'V26': [0.443210], 'V27': [0.332109], 'V28': [0.221098], 'Monto': [0.110987]
}, index=[0])

#Ver el registro
nuevo_registro

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Monto
0,0.000006,0.452345,0.564789,0.123456,0.654321,0.987654,0.345678,0.234567,0.876543,0.456789,...,0.109876,0.098765,0.887654,0.776543,0.665432,0.554321,0.44321,0.332109,0.221098,0.110987


In [12]:
#6.1. Realizar la predicción
clase_predicha=bosque.predict(nuevo_registro)
clase_predicha #La operación es Legitima o no
#  0 (ope. legitima)    1 (Fraudulenta)

array([0.])

In [13]:
#6.2. Probabilidad si la transacción es legitima
probabilidad = bosque.predict_proba(nuevo_registro)
probabilidad #% de probab. si es legitimo o no

array([[0.65, 0.35]])

In [14]:
#6.3. Personalizar la salida
print("Clase Predicha:", clase_predicha[0])
print("Probabilidad de Legitimidad:",probabilidad[0][0])
print("Probabilidad de Fraude:", probabilidad[0][1])

Clase Predicha: 0.0
Probabilidad de Legitimidad: 0.65
Probabilidad de Fraude: 0.35
